<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/branch_felix/IMECOS_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# initialize the database, see import statements in *implementation.py
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/src/source_database_implementation.py').text)

In [ ]:
# decions about the design of the star scheme have followed several deeper looks into the data. Included here for documentation purpose.
# EXECUTION OF THIS CELL IS OPTIONAL!
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/data_clearing_SQL_queries.py').text)

In [ ]:
data_clearing_queries(30)

In [ ]:
# ETL process
# CREATE VIEW for central table  (but with DESCRIPTIONS)
cur.executescript(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/ETL_create_view_central.sql').text)
conn.commit()
# Perform Extraction, Transformation and Loading into new database
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/ETL_process.py').text)

Data analysis

In [9]:
# import plot libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# save tables from DB into dataframe
encounters = pd.read_sql_query("SELECT * FROM encounters", conn_new)
patients = pd.read_sql_query("SELECT * FROM patients", conn_new)

In [ ]:
# Descriptive statistics PATIENTS
patients.info()

In [12]:
# Convert date columns
patients["BIRTHDATE"] = pd.to_datetime(patients["BIRTHDATE"])
patients["DEATHDATE"] = pd.to_datetime(patients["DEATHDATE"])
# calcute patient age
patients["AGE"] = patients.DEATHDATE.dt.year - patients.BIRTHDATE.dt.year
patients["AGE"] = patients["AGE"].fillna(2021 - patients.BIRTHDATE.dt.year)

In [ ]:
# patients age distribution
sns.displot(patients["AGE"])

In [ ]:
# patients marital
sns.countplot(x="variable", hue= "value", data=pd.melt(patients[["MARITAL"]]))


In [ ]:
# patients race
sns.countplot(x="variable", hue= "value", data=pd.melt(patients[["RACE"]]))

In [ ]:
# patients ethnicity
sns.countplot(x="variable", hue= "value", data=pd.melt(patients[["ETHNICITY"]]))

In [ ]:
# patients gender
sns.countplot(x="variable", hue= "value", data=pd.melt(patients[["GENDER"]]))

In [ ]:
# patients COV_EXP_RATIO - REMOVE OUTLIERS
sns.boxplot(x=patients["COV_EXP_RATIO"])

In [18]:
patients = patients[patients["COV_EXP_RATIO"]<=1]

In [ ]:
# COV_EXP_RATIO / age
sns.displot(x=patients["COV_EXP_RATIO"], y=patients["AGE"])

In [ ]:
# COV_EXP_RATIO / marital
sns.displot(x=patients.dropna()["COV_EXP_RATIO"], y=patients.dropna()["MARITAL"])

In [ ]:
# COV_EXP_RATIO / age
sns.displot(x=patients["COV_EXP_RATIO"], y=patients["RACE"])

In [ ]:
# COV_EXP_RATIO / age
sns.displot(x=patients["COV_EXP_RATIO"], y=patients["ETHNICITY"])

In [ ]:
# COV_EXP_RATIO / age
sns.displot(x=patients["COV_EXP_RATIO"], y=patients["GENDER"])

In [23]:
# convert categorical variable to numerical by replacing
patients.GENDER.replace(['M', 'F'], [0, 1], inplace=True)
patients.MARITAL.replace(['M', 'S'], [0, 1], inplace=True)
patients.RACE.replace(["white", "black", "asian", "native", "other"], [1, 2, 3, 4, 5], inplace=True)
patients.ETHNICITY.replace(["nonhispanic", "hispanic"], [0, 1], inplace=True)

In [ ]:
# Generating the correlating matrix
corr = patients.corr()
plt.figure(figsize=(7, 5))
# Generating the correlation heat-map

sns.heatmap(corr, annot=False)

In [25]:
from sklearn.preprocessing import StandardScaler


features = ['RACE', 'ETHNICITY', 'GENDER', 'AGE']
# # Separating out the features
x = patients.loc[:, features].values
# # Separating out the target
y = np.round(patients.loc[:,['COV_EXP_RATIO']].values, 1)
# # Standardizing the features
x = StandardScaler().fit_transform(x)

# check normalization
x.shape, np.mean(x), np.std(x)

((7352, 4), 8.45654099279227e-18, 1.0)

In [ ]:
# patients RACE after normalization
sns.countplot(x="variable", hue= "value", data=pd.melt(pd.DataFrame(x)[[0]]))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, patients[['COV_EXP_RATIO']].round(1)], axis = 1)
finalDf.head()

In [ ]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
finalDf['COV_EXP_RATIO'].unique()

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
#targets = ['RACE', 'ETHNICITY','GENDER', 'AGE']
targets = [0, 0.4]
colors = ['r', 'g', 'b', 'c', 'y']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['COV_EXP_RATIO'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
# Descriptive statistics encounters
encounters.info()


In [ ]:
# distribution ENCOUNTERCLASS
sns.countplot(x="variable", hue= "value", data=pd.melt(encounters.groupby('Id').agg(np.unique)[['ENCOUNTERCLASS']]))


In [ ]:
# distribution DATASET_ORIGIN
sns.countplot(x="variable", hue= "value", data=pd.melt(encounters.groupby('Id').agg(np.unique)[['DATASET_ORIGIN']]))
plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)